In [1]:
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
import keras
import keras.layers as layers
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import RepeatedKFold, train_test_split
from sklearn.metrics import confusion_matrix

2024-05-25 20:42:15.841493: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-25 20:42:16.373437: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
DataSolarModules = pd.read_json('InfraredSolarModules/module_metadata.json').transpose().sort_index()
Classes = DataSolarModules['anomaly_class'].unique()
class_to_number = dict(enumerate(Classes.flatten(), 0))
class_to_number = {v: k for k, v in class_to_number.items()}

def map_to_class(value):
    class_to_number
    return class_to_number.get(value, 'Unknown')

DataSolarModules['class_code'] = DataSolarModules['anomaly_class'].apply(map_to_class)

def read_images_dataframe(dataframe):
    images = []
    for image_path in dataframe['image_filepath']:
        img = cv2.imread("InfraredSolarModules/"+image_path,cv2.IMREAD_GRAYSCALE)
        img = img.reshape(40, 24).astype("float32") / 255
        images.append(img)
    images=np.array(images) 
    return images

def read_labels_dataframe(dataframe):
    labels = []
    for label in dataframe['class_code']:
        labels.append(label)
    labels=np.array(labels) 
    labels.astype("int32")   
    return labels

In [3]:
images = read_images_dataframe(DataSolarModules)
labels = read_labels_dataframe(DataSolarModules)

In [4]:
# Create a testing set with 25 images per class
num_images_per_class = 40

def create_testing_set(dataframe, num_images_per_class):
    test_indices = []
    for cls in dataframe['class_code'].unique():
        class_indices = dataframe[dataframe['class_code'] == cls].index[:num_images_per_class].tolist()
        test_indices.extend(class_indices)
    return test_indices

test_indices = create_testing_set(DataSolarModules, num_images_per_class)
data_test = np.array(test_indices)

# Create a training set with the remaining images
data_train = np.setdiff1d(np.arange(len(images)), data_test)

In [5]:
images_train = images[data_train]
labels_train = labels[data_train]

In [6]:
def apply_geometric_transformations(images_train, labels_train):
    transformed_images = []
    transformed_labels = []

    for i in range(len(images)):
        original_image = images[i]
        original_label = labels[i]

        # Append original image and label
        transformed_images.append(original_image)
        transformed_labels.append(original_label)

        # Flip vertically
        flipped_vertically = cv2.flip(original_image, 0)
        transformed_images.append(flipped_vertically)
        transformed_labels.append(original_label)

        # Flip horizontally
        flipped_horizontally = cv2.flip(original_image, 1)
        transformed_images.append(flipped_horizontally)
        transformed_labels.append(original_label)

        # Flip both vertically and horizontally
        flipped_both = cv2.flip(original_image, -1)
        transformed_images.append(flipped_both)
        transformed_labels.append(original_label)

    transformed_images = np.array(transformed_images) * 255  # Convert back to 0-255 range
    transformed_labels = np.array(transformed_labels)
    return transformed_images, transformed_labels

# Apply geometric transformations
images_GT, labels_GT = apply_geometric_transformations(images, labels)


In [7]:
import os
import json

# Create directory for saving images and metadata
os.makedirs("InfraredSolarModules/imagesGT", exist_ok=True)

# Define metadata path
metadata_path = "InfraredSolarModules/GT_metadata.json"

# Define function to save images and generate metadata
def save_images_and_metadata(images, labels, metadata_path):
    metadata = {}
    for i in range(len(images)):
        # Save image
        image_name = f"{i}.jpg"
        save_path = os.path.join("InfraredSolarModules/imagesGT", image_name)
        cv2.imwrite(save_path, images[i])

        # Generate metadata entry
        metadata_entry = {
            "image_filepath": f"imagesGT/{image_name}",
            "anomaly_class": Classes[int(labels[i])]
        }
        metadata[str(i)] = metadata_entry

    # Write metadata to JSON file
    with open(metadata_path, "w") as f:
        json.dump(metadata, f)

# Save images and metadata
save_images_and_metadata(images_GT, labels_GT, metadata_path)